In [1]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import tensorflow as tf
from tools import tools
from tools import Preprocessing as pp
from models import alexnet

import tensorflow as tf

%matplotlib inline

import os
# GPU Setting
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="2,3"

Classes=np.array(["Right ankle", "Right knee", "Right hip", "Left hip", "Left knee", "Left ankle", "Right wrist",
                  "Right elbow", "Right shoulder", "Left shoulder", "Left elbow", "Left wrist", "Neck", "Head top"])

In [2]:
testset=pp.config(pp.csv_file)
img_set,coor_set=pp.prepare_dataset.prepare(testset)

/home/hahnz/.local/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [3]:
X = tf.placeholder(tf.float32, [None, 256,256,3])
y = tf.placeholder(tf.float32, [None, 14])

In [4]:
alexnet = alexnet.config("alexnet", X, y)